# Global probing performance by triplet count

This notebook plots the non-CLS global probe results by triplet count. The y-axis uses the false positive rate reported in `probe_results.json`, which is already normalized by the number of absent pairs in each group (mean over absent labels).

In [ ]:
import json
import math
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

RESULTS_PATHS = [
    Path("output/probing_global/scene_description_balanced_2d_qwen2.5-VL-7B-Instruct/probes/probe_results.json"),
    Path("output/probing_global/scene_description_balanced_2d_qwen3-vl-30b-a3b-instruct/probes/probe_results.json"),
]

USE_STAR_MARKERS = False

for p in RESULTS_PATHS:
    if not p.exists():
        raise FileNotFoundError(f"Missing results file: {p}")

def _load_results(path: Path):
    raw = path.read_text()
    results = json.loads(raw, parse_constant=lambda x: float("nan"))
    non_cls_keys = [k for k in results.keys() if k.startswith("X_") and not k.startswith("X_cls_")]
    non_cls_keys = sorted(non_cls_keys, key=lambda k: int(k.split("_", 1)[1]))
    return results, non_cls_keys

def _series_from_group(d):
    xs = []
    ys = []
    for k, v in d.items():
        x = int(k)
        y = float(v)
        if math.isnan(y):
            y = np.nan
        xs.append(x)
        ys.append(y)
    order = np.argsort(xs)
    xs = np.asarray(xs)[order]
    ys = np.asarray(ys)[order]
    return xs, ys

fig, axes = plt.subplots(len(RESULTS_PATHS), 2, figsize=(12, 4 * len(RESULTS_PATHS)), sharey=True)
if len(RESULTS_PATHS) == 1:
    axes = np.array([axes])

for row, path in enumerate(RESULTS_PATHS):
    results, non_cls_keys = _load_results(path)
    model_name = path.parents[1].name

    for key in non_cls_keys:
        layer = int(key.split("_", 1)[1])
        implied = results[key]["implied_fpr_by_triplet"]
        non_implied = results[key]["non_implied_fpr_by_triplet"]

        x_i, y_i = _series_from_group(implied)
        x_n, y_n = _series_from_group(non_implied)

        marker_style = "*" if USE_STAR_MARKERS else "o"
        axes[row, 0].plot(x_i, y_i, marker=marker_style, linewidth=1.5, label=f"layer {layer}")
        axes[row, 1].plot(x_n, y_n, marker=marker_style, linewidth=1.5, label=f"layer {layer}")

    axes[row, 0].set_title(f"{model_name} - Implied-absent FPR")
    axes[row, 1].set_title(f"{model_name} - Non-implied-absent FPR")
    for col in range(2):
        ax = axes[row, col]
        ax.set_xlabel("Triplet count")
        ax.set_ylabel("False positive rate (normalized by absent pairs)")
        ax.grid(True, alpha=0.3)
        ax.legend(title="Layer", fontsize=8)

plt.tight_layout()

out_dir = Path("analysis/plots")
out_dir.mkdir(parents=True, exist_ok=True)
base_name = "global_probe_fpr_by_triplet"
for ext in ("pdf", "png", "svg"):
    fig.savefig(out_dir / f"{base_name}.{ext}", bbox_inches="tight")

plt.show()
